In [23]:
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

In [24]:
url = "bolt://localhost:7687"
username = "neo4j"
password= "neo4j"

driver = GraphDatabase.driver(url, auth=(username, password))

In [25]:
def queryNeo4j(driver, query):
    """Runs a single Cypher query."""
    with driver.session() as session:
        try:
            session.run(query)
            # print("Query executed successfully.")
        except Exception as e:
            print(f"Error executing query: {e}")

In [26]:
#deletes all nodes
queryNeo4j(driver , "MATCH(n) DETACH DELETE n;")

In [ ]:
#dels constraint
queryNeo4j(driver , "DROP CONSTRAINT n10s_unique_uri;")

In [28]:
def create_constraint(driver):
    # Create a session
    with driver.session() as session:
        # Run the updated Cypher query to create the constraint
        cypher_query = """CREATE CONSTRAINT n10s_unique_uri FOR (r:Resource) REQUIRE r.uri IS UNIQUE;"""
        session.run(cypher_query)
        print("Constraint created successfully.")

# Call the function to create the constraint
create_constraint(driver)

Constraint created successfully.


In [29]:
queryNeo4j(driver, """CALL n10s.graphconfig.init({
    handleVocabUris: "SHORTEN",
    keepLangTag: false,
    handleMultival: "ARRAY"
});   
           """)

In [30]:
# original annetto
# queryNeo4j(driver, "CALL n10s.rdf.import.fetch(\"https://raw.githubusercontent.com/iaklampanos/annett-o/refs/heads/master/annett-o-0.1.owl\",\"RDF/XML\");")

#github
queryNeo4j(driver, "CALL n10s.rdf.import.fetch(\"https://github.com/tbraunny/Automatic_Taxonomy_Construction_NNs/blob/f5c03faf4eaed4287cdb6729eca63a435bbf1525/data/owl/annett-o-test.owl\",\"RDF/XML\");")

#local
# queryNeo4j(driver, "CALL n10s.rdf.import.fetch(\"file:///home/lukas/CS425/Automatic_Taxonomy_Construction_NNs/data/annett-o-test.owl\",\"RDF/XML\");")



In [34]:
#strips URI
queryNeo4j(driver, """
MATCH (n)
WHERE n.uri STARTS WITH 'http://w3id.org/annett-o/'
SET n.uri = SPLIT(n.uri, '/')[SIZE(SPLIT(n.uri, '/')) - 1]
""")

In [36]:
#makes taxonomy
queryNeo4j(driver, """
MERGE (taskType_Taxon:Taxonomy {name: 'TaskType', type: 'CategoryTask'})
MERGE (FullyConnectedLayerType_Taxon:Taxonomy {name: 'LayerType', type: 'CategoryLayer', LayerTypes: 'FullyConnected'})
MERGE (CompositeLayerType_B_C_D_FC_Taxon:Taxonomy {name: 'LayerType', type: 'CategoryLayer', LayerTypes: 'BatchNorm, Concat, Dropout, FullyConnected'})
MERGE (GAN_Taxon:Taxonomy {name: 'GAN', type: 'Network', task: 'Adversarial, Classification, Generative', LayerTypes: 'FullyConnected'})
MERGE (AAE_Taxon:Taxonomy {name: 'AAE', type: 'Network', task: 'Adversarial, Clustering, Discrimination, Generation, Reconstruction', LayerTypes: 'BatchNorm, Concat, Dropout, FullyConnected'})
MERGE (simpleClassification_Taxon:Taxonomy {name: 'SimpleClassification', type: 'Network', task: 'SupervisedClassification', LayerTypes: 'FullyConnected'})
MERGE (taskType_Taxon)-[:hasAdversarial]->(FullyConnectedLayerType_Taxon)
MERGE (taskType_Taxon)-[:hasClassification]->(FullyConnectedLayerType_Taxon)
MERGE (taskType_Taxon)-[:hasGenerative]->(FullyConnectedLayerType_Taxon)
MERGE (taskType_Taxon)-[:hasSupervisedClassification]->(FullyConnectedLayerType_Taxon)
MERGE (taskType_Taxon)-[:hasAdversarial]->(CompositeLayerType_B_C_D_FC_Taxon)
MERGE (taskType_Taxon)-[:hasClustering]->(CompositeLayerType_B_C_D_FC_Taxon)
MERGE (taskType_Taxon)-[:hasDiscrimination]->(CompositeLayerType_B_C_D_FC_Taxon)
MERGE (taskType_Taxon)-[:hasGeneration]->(CompositeLayerType_B_C_D_FC_Taxon)
MERGE (taskType_Taxon)-[:hasReconstruction]->(CompositeLayerType_B_C_D_FC_Taxon)
MERGE (CompositeLayerType_B_C_D_FC_Taxon)-[:hasBatchNorm]->(AAE_Taxon)
MERGE (CompositeLayerType_B_C_D_FC_Taxon)-[:hasConcat]->(AAE_Taxon)
MERGE (CompositeLayerType_B_C_D_FC_Taxon)-[:hasDropout]->(AAE_Taxon)
MERGE (CompositeLayerType_B_C_D_FC_Taxon)-[:hasFullyConnected]->(AAE_Taxon)
MERGE (FullyConnectedLayerType_Taxon)-[:hasFullyConnected]->(GAN_Taxon)
MERGE (FullyConnectedLayerType_Taxon)-[:hasFullyConnected]->(simpleClassification_Taxon)
""")

In [39]:
from yfiles_jupyter_graphs import GraphWidget
def showGraph():
    session = driver.session()
    widget = GraphWidget(graph = session.run("MATCH (n)-[r]->(m) RETURN n, r, m").graph())
    widget.node_label_mapping = 'uri'
    widget.circular_layout()
    widget.set_sidebar(start_with='Data')
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
MATCH(n) DETACH DELETE n;

DROP CONSTRAINT n10s_unique_uri;

CREATE CONSTRAINT n10s_unique_uri FOR (r:Resource) REQUIRE r.uri IS UNIQUE;

CALL n10s.graphconfig.init({
    handleVocabUris: "SHORTEN",
    keepLangTag: false,
    handleMultival: "ARRAY",
    'n10s.rdf.import.fetch': 'file:///home/lukas/CS425/Automatic_Taxonomy_Construction_NNs/data/Annett-o/annett-o-test2.owl'
});

CALL n10s.rdf.import.fetch("https://raw.githubusercontent.com/tbraunny/Automatic_Taxonomy_Construction_NNs/f5c03faf4eaed4287cdb6729eca63a435bbf1525/data/owl/annett-o-test.owl", "RDF/XML");


CALL n10s.rdf.import.fetch("file:///home/lukas/CS425/Automatic_Taxonomy_Construction_NNs/data/annett-o-test.owl", "RDF/XML");

CALL n10s.rdf.import.fetch("https://raw.githubusercontent.com/iaklampanos/annett-o/refs/heads/master/annett-o-0.1.owl", "RDF/XML");

In [ ]:
cypher-shell -u neo4j -p neo4j -a bolt://localhost:7687